In [1]:
#TODO: tune BE

In [2]:
import os
import time
import numpy as np
import pandas as pd

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, ShuffleSplit, RandomizedSearchCV, KFold
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics
from sklearn import linear_model
from sklearn import svm
from xgboost.sklearn import XGBRegressor

In [4]:
def rmsle(h, y):
    return np.sqrt(np.square(np.log(h + 1) - np.log(y + 1)).mean())
rmsle_scorer = metrics.make_scorer(rmsle, greater_is_better=False)

In [5]:
df_train = pd.read_csv("../data/train_new.csv")

In [6]:
dist_features = [x for x in df_train.columns.tolist() if x.startswith('dist')]
path_features = [x for x in df_train.columns.tolist() if x.startswith('path')]
angle_features = [x for x in df_train.columns.tolist() if x.startswith('angle')]
dihedral_features = [x for x in df_train.columns.tolist() if x.startswith('dihedral')]
percent_features = [x for x in df_train.columns.tolist() if x.startswith('percent')]
avg_features = [x for x in df_train.columns.tolist() if x.startswith('avg')]
lattice_angle_features = [x for x in df_train.columns.tolist() if x.startswith('lattice_angle')]

In [ ]:
# when labels are in [0,1] we can also use reg:logistic
# Create a parameter grid to search for best parameters for everything in the pipeline
param_grid = {
                #'min_child_weight': [0.01,0.05,0.1, 0.5,1,1.5,2], # due to high class imbalance
                'booster':['gbtree'],
                'objective': ['reg:logistic','reg:linear'],
                'max_depth': [4,5,6,7],
                'nthread': [12],
                #'max_delta_step': [1.8],
                #'colsample_bytree': [0.4],
                'subsample': [0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 0.99],
                'learning_rate': [0.01,0.075,0.9,0.1,0.012,0.013,0.014,0.015], #0.025
                'gamma': [0.01,0.011,0.009,0.008,0.012,0.015,0.02,0.005],
                'n_estimators' : [250],
                #'tree_method': ['gpu_hist'],
                'silent': [1],
                'missing':[-999],
                'n_jobs': [12],
                #'updater': 'grow_gpu',
                #'scale_pos_weight' : [0.5,1,1.5] # due to high class imbalance
              }

# Normalized Gini Scorer
rmsle_scorer = metrics.make_scorer(rmsle, greater_is_better=False)

# Initialize Grid Search Model
model = RandomizedSearchCV(estimator  = XGBRegressor(),
                           param_distributions = param_grid,
                           n_iter = 20,
                           #param_grid = param_grid,
                           scoring    = rmsle_scorer,
                           verbose    = 2,
                           n_jobs     = 12,
                           iid        = True,
                           refit      = True,
                           cv         = KFold(n_splits=10, shuffle=False, random_state=42))
# Fit Grid Search Model
features_be = ['lattice_angle_alpha_degree',
               'lattice_angle_gamma_degree',
               'lattice_angle_gamma_degree',
               'number_of_total_atoms',
               'vol',
               'atomic_density',
               'lattice_vector_1_ang',
               'lattice_vector_2_ang',
               'lattice_vector_3_ang']+avg_features+percent_features+dist_features+angle_features+dihedral_features



X = df_train[features_be].values
y_be = np.log(df_train['bandgap_energy_ev'].values +1)

scaler_be = MinMaxScaler().fit(y_be.reshape(-1, 1))
y_be = scaler_be.transform(y_be.reshape(-1, 1)).flatten()

model.fit(X, y_be)
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

# Get best model
best_model = model.best_estimator_

# Fit model with best parameters optimized for normalized_gini
#best_model.fit(train,labels)

Fitting 10 folds for each of 20 candidates, totalling 200 fits
[CV] subsample=0.9, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=5, learning_rate=0.2, gamma=0.015, booster=gbtree 
[CV] subsample=0.9, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=5, learning_rate=0.2, gamma=0.015, booster=gbtree 
[CV] subsample=0.9, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=5, learning_rate=0.2, gamma=0.015, booster=gbtree 
[CV] subsample=0.9, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=5, learning_rate=0.2, gamma=0.015, booster=gbtree 
[CV] subsample=0.9, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=5, learning_rate=0.2, gamma=0.015, booster=gbtree 
[CV] subsample=0.9, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, miss

[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed: 10.2min


[CV]  subsample=0.8, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=5, learning_rate=0.3, gamma=0.015, booster=gbtree, total= 5.2min
[CV] subsample=0.7, silent=1, objective=reg:logistic, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=6, learning_rate=0.2, gamma=0.015, booster=gbtree 
[CV]  subsample=0.8, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=5, learning_rate=0.3, gamma=0.015, booster=gbtree, total= 5.6min
[CV]  subsample=0.8, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=5, learning_rate=0.3, gamma=0.015, booster=gbtree, total= 5.4min
[CV] subsample=0.8, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=5, learning_rate=0.1, gamma=0.009, booster=gbtree 
[CV] subsample=0.8, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_de

[CV] subsample=0.7, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=5, learning_rate=0.5, gamma=0.009, booster=gbtree 
[CV]  subsample=0.95, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=5, learning_rate=0.1, gamma=0.008, booster=gbtree, total= 5.5min
[CV] subsample=0.7, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=5, learning_rate=0.5, gamma=0.009, booster=gbtree 
[CV]  subsample=0.95, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=5, learning_rate=0.1, gamma=0.008, booster=gbtree, total= 5.4min
[CV] subsample=0.7, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=5, learning_rate=0.5, gamma=0.009, booster=gbtree 
[CV]  subsample=0.95, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=5, learni

[CV]  subsample=0.99, silent=1, objective=reg:logistic, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=6, learning_rate=0.5, gamma=0.008, booster=gbtree, total= 6.4min
[CV] subsample=0.8, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=5, learning_rate=0.4, gamma=0.01, booster=gbtree 
[CV]  subsample=0.8, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=5, learning_rate=0.4, gamma=0.01, booster=gbtree, total= 5.6min
[CV] subsample=0.8, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=5, learning_rate=0.4, gamma=0.01, booster=gbtree 
[CV]  subsample=0.8, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=5, learning_rate=0.4, gamma=0.01, booster=gbtree, total= 5.7min
[CV] subsample=0.8, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth

[CV] subsample=0.95, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=4, learning_rate=0.3, gamma=0.02, booster=gbtree 
[CV]  subsample=0.95, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=4, learning_rate=0.3, gamma=0.02, booster=gbtree, total= 3.9min
[CV] subsample=0.8, silent=1, objective=reg:logistic, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=5, learning_rate=0.5, gamma=0.02, booster=gbtree 
[CV]  subsample=0.95, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=4, learning_rate=0.3, gamma=0.02, booster=gbtree, total= 3.6min
[CV] subsample=0.8, silent=1, objective=reg:logistic, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=5, learning_rate=0.5, gamma=0.02, booster=gbtree 
[CV]  subsample=0.75, silent=1, objective=reg:logistic, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=5, lear

[CV]  subsample=0.85, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=7, learning_rate=0.4, gamma=0.015, booster=gbtree, total= 9.1min
[CV] subsample=0.85, silent=1, objective=reg:logistic, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=5, learning_rate=0.5, gamma=0.02, booster=gbtree 
[CV]  subsample=0.85, silent=1, objective=reg:logistic, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=5, learning_rate=0.5, gamma=0.02, booster=gbtree, total= 5.1min
[CV] subsample=0.85, silent=1, objective=reg:logistic, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=5, learning_rate=0.5, gamma=0.02, booster=gbtree 
[CV]  subsample=0.85, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=7, learning_rate=0.4, gamma=0.015, booster=gbtree, total= 8.6min
[CV] subsample=0.85, silent=1, objective=reg:logistic, nthread=12, n_jobs=12, n_estimators=250, missing=-99

[CV] subsample=0.7, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=6, learning_rate=0.4, gamma=0.009, booster=gbtree 
[CV]  subsample=0.99, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=5, learning_rate=0.2, gamma=0.012, booster=gbtree, total= 5.5min
[CV] subsample=0.7, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=6, learning_rate=0.4, gamma=0.009, booster=gbtree 
[CV]  subsample=0.99, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=5, learning_rate=0.2, gamma=0.012, booster=gbtree, total= 5.4min
[CV] subsample=0.7, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=6, learning_rate=0.4, gamma=0.009, booster=gbtree 
[CV]  subsample=0.99, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=5, learni

[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed: 66.8min


[CV]  subsample=0.99, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=5, learning_rate=0.2, gamma=0.012, booster=gbtree, total= 5.1min
[CV] subsample=0.7, silent=1, objective=reg:logistic, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=4, learning_rate=0.5, gamma=0.009, booster=gbtree 
[CV]  subsample=0.99, silent=1, objective=reg:linear, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=5, learning_rate=0.2, gamma=0.012, booster=gbtree, total= 6.1min
[CV] subsample=0.7, silent=1, objective=reg:logistic, nthread=12, n_jobs=12, n_estimators=250, missing=-999, max_depth=4, learning_rate=0.5, gamma=0.009, booster=gbtree 


In [ ]:
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

# Get best model
best_model = model.best_estimator_